# DeepStream YOLOv13 Object Detection

This notebook runs **YOLOv13** detection using [DeepStream-Yolo](https://github.com/marcoslucianops/DeepStream-Yolo) with NVIDIA DeepStream SDK.

**Run this notebook inside the DeepStream container** (e.g. `manasi1096/deepstream8-python:h100`) where `/workspace` is mounted to your project directory.

## 1. Setup paths and install build tools

In [ ]:
import os
import sys
import shutil
from pathlib import Path

# Resolve project root — the notebook lives in DeepStream-Yolo-YOLOv13/
# Inside the container, /workspace is mounted from the host's /home/nvidia
NOTEBOOK_DIR = Path("/workspace/DeepStream-Yolo-YOLOv13")
if not NOTEBOOK_DIR.exists():
    NOTEBOOK_DIR = Path("/home/nvidia/DeepStream-Yolo-YOLOv13")
DS_YOLO_DIR = NOTEBOOK_DIR / "DeepStream-Yolo"

os.chdir(NOTEBOOK_DIR)

# Clone DeepStream-Yolo if not present
if not DS_YOLO_DIR.exists():
    ! git clone --depth 1 https://github.com/marcoslucianops/DeepStream-Yolo.git {DS_YOLO_DIR}

# Copy pre-made config files into DeepStream-Yolo
CONFIGS_DIR = NOTEBOOK_DIR / "configs"
if CONFIGS_DIR.exists():
    for cfg in CONFIGS_DIR.glob("*.txt"):
        shutil.copy2(cfg, DS_YOLO_DIR / cfg.name)
        print(f"Copied config: {cfg.name}")

print("Project root:", NOTEBOOK_DIR)
print("DeepStream-Yolo:", DS_YOLO_DIR)
assert DS_YOLO_DIR.exists(), "DeepStream-Yolo not found."

# Install build-essential if needed (for compiling nvdsinfer custom lib)
! apt-get update -qq && apt-get install -y -qq build-essential > /dev/null 2>&1
print("Build tools ready.")

## 2. Get YOLOv13 ONNX model

Clones the [iMoonLab/yolov13](https://github.com/iMoonLab/yolov13) repo, installs it as a package (it's a fork of ultralytics with custom YOLOv13 modules), downloads `yolov13s.pt`, and exports to ONNX using the DeepStream-Yolo converter.

In [ ]:
YOLOV13_DIR = NOTEBOOK_DIR / "yolov13"
YOLOV13_DIR.mkdir(exist_ok=True)
os.chdir(YOLOV13_DIR)

# Clone YOLOv13 repo (iMoonLab fork of ultralytics with custom arch)
if not (YOLOV13_DIR / "setup.py").exists() and not (YOLOV13_DIR / "pyproject.toml").exists():
    ! git clone --depth 1 https://github.com/iMoonLab/yolov13.git repo
    if (YOLOV13_DIR / "repo").exists():
        ! cp -r repo/* repo/.* . 2>/dev/null; rm -rf repo

# IMPORTANT: Install the yolov13 repo as a package — it is a fork of ultralytics
# with custom YOLOv13 modules. Vanilla ultralytics cannot load yolov13s.pt.
! pip install -e . 2>&1 | tail -5
! pip install -q onnx onnxslim onnxruntime onnxscript huggingface_hub

# Download YOLOv13s weights from releases
PT_FILE = YOLOV13_DIR / "yolov13s.pt"
if not PT_FILE.exists():
    ! wget -q https://github.com/iMoonLab/yolov13/releases/download/yolov13/yolov13s.pt -O yolov13s.pt
print("Weights downloaded:", PT_FILE.exists())

# Copy DeepStream export script and run ONNX export
# NOTE: Do NOT use --simplify; the simplifier corrupts the external-data ONNX.
#       Use opset 18 (torch 2.10+ requires it).
EXPORT_SCRIPT = DS_YOLO_DIR / "utils" / "export_yoloV13.py"
assert EXPORT_SCRIPT.exists(), f"Export script not found at {EXPORT_SCRIPT}"
! cp {EXPORT_SCRIPT} .
! python3 export_yoloV13.py -w yolov13s.pt --dynamic --opset 18

# The new PyTorch ONNX exporter stores weights externally (yolov13s.onnx.data).
# Inline them into a single file so TensorRT can parse it.
import onnx
model_onnx = onnx.load("yolov13s.onnx", load_external_data=True)
onnx.save(model_onnx, str(DS_YOLO_DIR / "yolov13s.onnx"))
print(f"Inlined ONNX saved: {(DS_YOLO_DIR / 'yolov13s.onnx').stat().st_size / 1024 / 1024:.1f} MB")

# Copy labels
if (YOLOV13_DIR / "labels.txt").exists():
    ! cp labels.txt {DS_YOLO_DIR}/
print("ONNX model and labels ready in DeepStream-Yolo.")

os.chdir(NOTEBOOK_DIR)

## 3. Build DeepStream custom inference lib

Compile `nvdsinfer_custom_impl_Yolo` for your CUDA/DeepStream version (DeepStream 8.0 → CUDA_VER=12.8 on x86).

In [ ]:
os.chdir(DS_YOLO_DIR)

# DeepStream 8.0 x86 = 12.8; for Jetson/other see README
os.environ["CUDA_VER"] = "12.8"
! make -C nvdsinfer_custom_impl_Yolo clean
! make -C nvdsinfer_custom_impl_Yolo

SO_PATH = DS_YOLO_DIR / "nvdsinfer_custom_impl_Yolo" / "libnvdsinfer_custom_impl_Yolo.so"
print("Lib built:", SO_PATH.exists())
os.chdir(NOTEBOOK_DIR)

## 4. Run DeepStream app with YOLOv13

Runs `deepstream-app` with the YOLOv13 config. Input is the default DeepStream sample video. Output is saved to `output_yolov13.mp4` (file sink, no display needed). First run may take several minutes to build the TensorRT engine.

In [ ]:
os.chdir(DS_YOLO_DIR)

APP_CONFIG = "deepstream_app_config_yolov13.txt"
assert Path(APP_CONFIG).exists(), f"{APP_CONFIG} not found in {DS_YOLO_DIR}"

print("Running: deepstream-app -c", APP_CONFIG)
print("First run can take 5–10+ min for TensorRT engine build.")
print("Output will be saved to: output_yolov13.mp4")
! deepstream-app -c {APP_CONFIG}

# Check output
output_file = DS_YOLO_DIR / "output_yolov13.mp4"
if output_file.exists():
    size_mb = output_file.stat().st_size / (1024 * 1024)
    print(f"\nOutput saved: {output_file} ({size_mb:.1f} MB)")
else:
    print("\nWarning: output file not found — check the logs above for errors.")

## Optional: Run on your own video

Edit `[source0]` in `DeepStream-Yolo/deepstream_app_config_yolov13.txt`:
- `uri=file:///path/to/your/video.mp4`

Then run the cell above again.